In [1]:
import tensorflow as tf
import numpy as np

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Basic RNNs

## static rnn

In [16]:
tf.reset_default_graph()
n_inputs=3
n_neurons=5

X0=tf.placeholder(tf.float32,[None,n_inputs])
X1=tf.placeholder(tf.float32,[None,n_inputs])

basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs,states=tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32)
Y0,Y1=output_seqs

X0_batch=np.random.randint(0,10,(4,3))
X1_batch=np.random.randint(0,10,(4,3))

init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    Y0_val,Y1_val=sess.run([Y0,Y1],feed_dict={X0:X0_batch,X1:X1_batch})

print(Y0_val,'\n\n',Y1_val)

[[-0.99995846  0.64994746  0.9477255   0.98284125  0.9999328 ]
 [-0.5907206   0.9928911  -0.99987364 -0.86659724  0.28669733]
 [-0.9777578   0.9997149  -0.99999285 -0.9932403  -0.9603141 ]
 [-0.999967    0.9822514  -0.5380911   0.41656542  0.98254174]] 

 [[-0.9999987   0.9948482  -0.6889066   0.64732695  0.9997804 ]
 [-0.90015143  0.99731237 -0.9998616  -0.870236    0.7393007 ]
 [-0.96263134  0.959434   -0.98530495 -0.07533193  0.5642953 ]
 [-0.9992089   0.75599885  0.06243335  0.98870724  0.9999998 ]]


In [19]:
import sys
sys.path.append('../handson-ml')
from tensorflow_graph_in_jupyter import show_graph
#show_graph(tf.get_default_graph())

## dynamic rnn

In [25]:
tf.reset_default_graph()
n_inputs=3
n_neurons=5
n_steps=2

X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs,states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

init=tf.global_variables_initializer()
X_batch = np.array([
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val=outputs.eval(feed_dict={X:X_batch})

print(outputs_val)

[[[-0.13739704  0.47404662  0.8220064   0.6159681  -0.6323763 ]
  [ 0.9999972   0.999962   -0.13603927  0.9997109   0.9146552 ]]

 [[ 0.9541514   0.9907224   0.89607644  0.980434   -0.45596564]
  [ 0.73192    -0.4265762   0.23776443  0.73539215  0.7067916 ]]

 [[ 0.99916536  0.9998782   0.9403324   0.9991788  -0.23459452]
  [ 0.99990886  0.9977797  -0.39854965  0.9975424   0.9768478 ]]

 [[ 0.9997699   0.99998426 -0.9995244   0.36805505  0.79117316]
  [ 0.9293771   0.9286477  -0.17988071  0.9236963   0.9489698 ]]]


# Training to Predict Time Series

In [2]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [10]:
tf.reset_default_graph()

n_steps=20
n_inputs=1
n_neurons=100
n_outputs=1

X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.float32,[None,n_steps,n_outputs])

#cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation=tf.nn.elu)
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)
outputs,states=tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

cell=tf.contrib.rnn.OutputProjectionWrapper(
tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation=tf.nn.elu),output_size=n_outputs)

learning_rate=1e-3
loss=tf.reduce_mean(tf.square(outputs-y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)

init=tf.global_variables_initializer()

n_iterations=1000
batch_size=50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch,y_batch=next_batch(batch_size,n_steps)
        training_op.run(feed_dict={X:X_batch,y:y_batch})
        if (iteration+1)%100==0 or iteration==0:
            print(iteration,'mse:',loss.eval(feed_dict={X:X_batch,y:y_batch}))

0 mse: 12.404406
99 mse: 0.47094506
199 mse: 0.13753077
299 mse: 0.05824591
399 mse: 0.05306262
499 mse: 0.047133744
599 mse: 0.051453795
699 mse: 0.054657504
799 mse: 0.050477643
899 mse: 0.044989754
999 mse: 0.04391702
